# Visualizing Google Forms Data with Seaborn

This is the second part of an article from [Practical Business Python](htp://pbpython.com) describing how to retrieve and analyze data from a Google Form.

Please review [part 1](http://pbpython.com/pandas-google-forms-part1.html) for the details of how to set up authentication and get the data into the pandaqs dataframe.

The full article corresponding to this notebook is [here](http://pbpython.com/pandas-google-forms-part2.html)

## Setup

Bring in our standard imports as well as the authentication libraries we will need to get access to our form.

In [1]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install oauth2client

  Using cached httplib2-0.20.4-py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gspread
from oauth2client.client import SignedJwtAssertionCredentials
import pandas as pd
import json

ImportError: cannot import name 'SignedJwtAssertionCredentials' from 'oauth2client.client' (C:\Users\cti110016\Anaconda3\lib\site-packages\oauth2client\client.py)

Import Ipython display as well as graphing libraries. For this article, we will be using [seaborn](http://stanford.edu/~mwaskom/software/seaborn/index.html).

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Setup authentication process to pull in the survey data stored in the Google Sheet.

In [ ]:
SCOPE = ["https://spreadsheets.google.com/feeds"]
SECRETS_FILE = "Pbpython-key.json"
SPREADSHEET = "PBPython User Survey (Responses)"
# Based on docs here - http://gspread.readthedocs.org/en/latest/oauth2.html
# Load in the secret JSON key (must be a service account)
json_key = json.load(open(SECRETS_FILE))
# Authenticate using the signed key
credentials = SignedJwtAssertionCredentials(json_key['client_email'],
                                            json_key['private_key'], SCOPE)

Now open up the file and read all data in a DataFrame

In [ ]:
gc = gspread.authorize(credentials)
# Open up the workbook based on the spreadsheet name
workbook = gc.open(SPREADSHEET)
# Get the first sheet
sheet = workbook.sheet1
# Extract all data into a dataframe
results = pd.DataFrame(sheet.get_all_records())
results.head()

We need to do some cleanup to make the data easier to analyze.

In [ ]:
# Do some minor cleanups on the data
# Rename the columns to make it easier to manipulate
# The data comes in through a dictionary so we can not assume order stays the
# same so must name each column
column_names = {'Timestamp': 'timestamp',
                'What version of python would you like to see used for the examples on the site?': 'version',
                'How useful is the content on practical business python?': 'useful',
                'What suggestions do you have for future content?': 'suggestions',
                'How frequently do you use the following tools? [Python]': 'freq-py',
                'How frequently do you use the following tools? [SQL]': 'freq-sql',
                'How frequently do you use the following tools? [R]': 'freq-r',
                'How frequently do you use the following tools? [Javascript]': 'freq-js',
                'How frequently do you use the following tools? [VBA]': 'freq-vba',
                'How frequently do you use the following tools? [Ruby]': 'freq-ruby',
                'Which OS do you use most frequently?': 'os',
                'Which python distribution do you primarily use?': 'distro',
                'How would you like to be notified about new articles on this site?': 'notify'
                }
results.rename(columns=column_names, inplace=True)
results.timestamp = pd.to_datetime(results.timestamp)
results.head()

There are a small number of free form comments. Let's strip those out and remove them from the results.

In [ ]:
suggestions = results[results.suggestions.str.len() > 0]["suggestions"]

Since there are only a small number of comments, just print them out.
However, if we had more comments and wanted to do more analysis we certainly good.

In [ ]:
for index, row in suggestions.iteritems():
    display(row)

Drop the suggestions. We won't use them any more.

In [ ]:
results.drop("suggestions", axis=1, inplace=True)
results.head()

## Explore the data

For Numeric columns, start with describe to see what we have

In [ ]:
results.describe()

Because we only have 1, 2, 3 as options the numeric results aren't telling us that much. I am going to convert the number to more useful descriptions.

In [ ]:
results['useful'] = results['useful'].map({1: '1-low', 2: '2-medium', 3: '3-high'})
results.head()

Value counts give us an easy distribution view into the raw numbers

In [ ]:
results["version"].value_counts()

Use normalize to see it by percentage.

In [ ]:
results.os.value_counts(normalize=True)

While the numbers are useful, wouldn't it be nicer to visually show the results?

Seaborn's [factorplot](http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.factorplot.html) is helpful for showing this kind of categorical data.

Because factorplot is so powerful, I'll build up step by step to show how it can be used for complex data analysis.

First, look at number of users by OS.

In [ ]:
sns.factorplot("os", data=results, palette="BuPu")

It is easy to order the results using x_order

In [ ]:
sns.factorplot("os", x_order=["Linux", "Windows", "Mac"], data=results, palette="BuPu")

Do a similar plot on python version

In [ ]:
sns.factorplot("version", data=results, palette="BuPu")

This is useful but wouldn't it be better to compare with OS and preferred python version? This is where factorplot starts to show more versatility. The key component is to use hue to automatically slice the data by python version (in this case).

In [ ]:
sns.factorplot("os", hue="version", x_order=["Linux", "Windows", "Mac"], data=results, palette="Paired")

Because seaborn knows how to work with dataframes, we just need to pass in the column names for the various arguments and it will do the analysis and presentation.

How about if we try to see if there is any relationship between how useful the site is and OS/Python choice? We can add the useful column into the plot using col.

In [ ]:
sns.factorplot("version", hue="os", data=results, col="useful", palette="Paired")

If we can add a column, we can also add a row and seaborn takes care of the rest.

In looking at the data, we have two different versions of winpython so clean that up first.

In [ ]:
results['distro'] = results['distro'].str.replace('WinPython', 'winpython')

In [ ]:
results.head()

We can also look at the distros. Since there is some overlap with the distros and os, let's only look at a subset of distros. For instance, someone using winpython is not going to be using it on a Mac.

In [ ]:
results['distro'].value_counts()

The most meaningful data would be looking at the Anaconda and Official python.org binaries. Let's filter all of our data only on these two values.

In [ ]:
results_distro = results[results["distro"].isin(["Anaconda", "Official python.org binaries"])]
results_distro.head()

Now do our factorplot with multiple columns and rows using row and col.

In [ ]:
sns.factorplot("version", hue="os", data=results_distro, col="useful", row="distro", margin_titles=True, sharex=False)

## Responses over time

We know that we have 55 results now. It would be interesting to see how those results came in over time. Using this method, we can very simply look at this by any time period we want.

The seaborn's [timeseries](http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.tsplot.html) supports this type of analysis and much more.

For ease of calculating responses over time, add a count colum for each response.

In [ ]:
results["count"] = 1
results.head()

To get totals over time, set our index to the timestamp

In [ ]:
total_results = results.set_index('timestamp')
total_results.head()

Use pandas TimeGrouper to summarize the data by day and do a cumulative sum. We could easily do this for any time period too.

In [ ]:
running_results = total_results.groupby(pd.TimeGrouper('D'))["count"].count().cumsum()
running_results

To label the x-axis we need to define our time range

In [ ]:
step = pd.Series(range(0,len(running_results)), name="Days")
sns.tsplot(running_results, value="Total Responses", time=step, color="husl")

## Heatmaps and Clustermaps

The final section of data to analyze is the frequency with which readers are using different technology. I am going to use a [heatmap](http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.heatmap.html#seaborn.heatmap) to look for any interesting insights.

Let's look at the data again.

In [ ]:
results.head()

In [ ]:
results["freq-py"].value_counts()

What we need to do is construct a single DataFrame with all the value_counts for the specific technology.
First we will create a list containing each value count.

In [ ]:
all_counts = []
for tech in ["freq-py", "freq-sql", "freq-r", "freq-ruby", "freq-js", "freq-vba"]:
    all_counts.append(results[tech].value_counts())
display(all_counts)

Now, concat the lists along axis=1.

Fill in any nan values with 0 too.

In [ ]:
tech_usage = pd.concat(all_counts, keys=["Python", "SQL", "R", "Ruby", "javascript", "VBA"], axis=1)
tech_usage = tech_usage.fillna(0)
tech_usage

We have a nice table but there are a few problems.

First, we have one column with blank values that we don't want.

Secondly, we would like to order from Daily -> Never. Use reindex to accomplish both tasks.

In [ ]:
tech_usage = tech_usage.reindex(["Daily", "A couple times a week", "Once a month", "Infrequently", "Never"])
tech_usage

Now that the data is in the correct table format, we can create a heatmap.

In [ ]:
sns.heatmap(tech_usage, annot=True)

So, what does this tell us?

Not surprisingly, most people use python very frequently.

Additionally, it looks like very few survey takers are using Ruby or VBA.

A variation of the heatmap is the [clustermap](http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.clustermap.html#seaborn.clustermap). The main feature it does is that it tries to reorganize the data to more easily see relationships/clusters.

In [ ]:
sns.clustermap(tech_usage, annot=True)

At first glance, it may seem to be a repeat but you'll notice that the order of the axes are different.

For instance, python and SQL are clusterd in the lower right with higher usage and Ruby and VBA have a cluster in the upper left with lower usage.